In [0]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
 
spark = SparkSession.builder \
    .appName("IngestaoBronze") \
    .getOrCreate()

In [0]:
%run ./Intancia_Containers

In [0]:
# Lista para armazenar dicionários com os metadados
lista_arquivos_processada = []

for nome, obj in containers_registry.items():
    # Obtém a lista de arquivos do container
    arquivos = obj.get_files_list()
    
    for arq in arquivos:
        lista_arquivos_processada.append({
            "path": arq.path,
            "name": arq.name,
            "size": arq.size,
            "modificationTime": arq.modificationTime,
            "container_name": nome
        })

    print(f"✅ Metadados coletados de: {nome.upper()}")

# Criando o DataFrame com os arquivos
df_inventario = spark.createDataFrame(lista_arquivos_processada)

# Convertendo modificationTime para um formato mais amigável
df_inventario = df_inventario.withColumn(
    "modificationTime", 
    (F.col("modificationTime") / 1000).cast("timestamp")
)

display(df_inventario)

In [0]:
import zipfile
import os

def unzip_file(path_origem, container_nome):
    """
    Copia o ZIP do Blob Storage para o DBFS local e extrai os CSVs.
    """

    # Nome do arquivo
    zip_name = os.path.basename(path_origem)

    # Caminho temporário no DBFS
    dbfs_zip_path = f"dbfs:/tmp/{zip_name}"
    local_zip_path = f"/dbfs/tmp/{zip_name}"

    # Copia do Blob (wasbs) para DBFS
    dbutils.fs.cp(path_origem, dbfs_zip_path, recurse=False)

    # Pasta de extração
    extract_path = f"/dbfs/tmp/unzipped/{container_nome}"
    os.makedirs(extract_path, exist_ok=True)

    # Extrai os arquivos
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

        extracted_files = [
            f"/dbfs/tmp/unzipped/{container_nome}/{f}"
            for f in zip_ref.namelist()
            #if f.endswith(".csv")
        ]

    # Retorna caminhos compatíveis com Spark
    return [f.replace("/dbfs", "dbfs:") for f in extracted_files]

In [0]:
colunas_empresas = [
    "cnpj_basico",
    "razao_social",
    "natureza_juridica",
    "qualificacao_responsavel",
    "capital_social",
    "porte_empresa",
    "ente_federativo_responsavel"
]

colunas_estabelecimento = [
    "cnpj_basico",
    "cnpj_ordem",
    "cnpj_dv",
    "identificador_matriz_filial",
    "nome_fantasia",
    "situacao_cadastral",
    "data_situacao_cadastral",
    "motivo_situacao_cadastral",
    "nome_cidade_exterior",
    "pais",
    "data_inicio_atividade",
    "cnae_fiscal_principal",
    "cnae_fiscal_secundaria",
    "tipo_logradouro",
    "logradouro",
    "numero",
    "complemento",
    "bairro",
    "cep",
    "uf",
    "municipio",
    "ddd_1",
    "telefone_1",
    "ddd_2",
    "telefone_2",
    "ddd_fax",
    "fax",
    "email",
    "situacao_especial",
    "data_situacao_especial"
]

colunas_simples = [
    "cnpj_basico",
    "opcao_simples",
    "data_opcao_simples",
    "data_exclusao_simples",
    "opcao_mei",
    "data_opcao_mei",
    "data_exclusao_mei"
]

colunas_socios = [
    "cnpj_basico",
    "identificador_socio",
    "nome_socio_razao_social",
    "cpf_cnpj_socio",
    "qualificacao_socio",
    "data_entrada_sociedade",
    "pais",
    "cpf_representante_legal",
    "nome_representante_legal",
    "qualificacao_representante_legal",
    "faixa_etaria"
]

colunas_paises = [
    "codigo_pais",
    "descricao_pais"
]

colunas_municipios = [
    "codigo_municipio",
    "descricao_municipio"
]

colunas_qualificacoes = [
    "codigo_qualificacao",
    "descricao_qualificacao"
]

colunas_natureza = [
    "codigo_natureza_juridica",
    "descricao_natureza_juridica"
]

colunas_cnae = [
    "codigo_cnae",
    "descricao_cnae"
]

colunas_motivo = [
    "codigo_motivo",
    "descricao_motivo"
]

# Mapear nome para lista de colunas
colunas_map = {
    "Cnaes": colunas_cnae,
    "Empresas": colunas_empresas,
    "Estabelecimentos": colunas_estabelecimento,
    "Municipios": colunas_municipios,
    "Naturezas": colunas_natureza,
    "Paises": colunas_paises,
    "Qualificacoes": colunas_qualificacoes,
    "Simples": colunas_simples,
    "Socios": colunas_socios,
    "Motivos": colunas_motivo
}

In [0]:
def renomeia_colunas(df, nome_tabela):
    # Normaliza o nome (remove números como Empresas0, Empresas1)
    nome_normalizado = ''.join(filter(str.isalpha, nome_tabela))

    colunas_corretas = colunas_map.get(nome_normalizado)

    if not colunas_corretas:
        print(f"[Aviso] Nenhum dicionário encontrado para '{nome_tabela}'.")
        return df

    if len(df.columns) != len(colunas_corretas):
        print(
            f"[Aviso] '{nome_tabela}': número de colunas diferente "
            f"(DF={len(df.columns)} | Esperado={len(colunas_corretas)})."
        )
        return df

    # Renomeia todas as colunas de uma vez (mais performático)
    return df.toDF(*colunas_corretas)

In [0]:
import re

def nome_simples_tabela(nome_arquivo):
    base = nome_arquivo.split(".")[0]
    return re.sub(r"\d+$", "", base).lower()

In [0]:
def adiciona_metadados(df, path_origem):
    return (
        df.withColumn("_input_file_name", F.lit(path_origem))
          .withColumn("_ingestion_timestamp", F.current_timestamp())
    )

In [0]:
TGT_STORAGE_ACCOUNT = dbutils.secrets.get(scope="secrets-kv", key="tgt-storage-account")
TGT_CONTAINER = dbutils.secrets.get(scope="secrets-kv", key="tgt-container")
TGT_SAS_TOKEN = dbutils.secrets.get(scope="secrets-kv", key="tgt-sas-token")

# Configuring SAS for Lakehouse Container
spark.conf.set(
    f"fs.azure.sas.{TGT_CONTAINER}.{TGT_STORAGE_ACCOUNT}.blob.core.windows.net",
    TGT_SAS_TOKEN
)

LAKEHOUSE_BASE_PATH = f"wasbs://{TGT_CONTAINER}@{TGT_STORAGE_ACCOUNT}.blob.core.windows.net/bronze_1"

In [0]:
from functools import reduce

df_empresas = []
df_estabelecimentos = []
df_socios = []
df_outros = {}

In [0]:
from pyspark.sql.functions import col
from delta.tables import DeltaTable

# iterando nos Arquivos
for row in df_inventario.collect():
    path_origem = row['path']
    container = row['container_name']
    nome_arquivo = row['name']
    
    # Define o caminho de destino (removendo a extensão para o nome da tabela Delta)
    tabela_nome = nome_arquivo.split('.')[0]    
    print(f"⏳ Processando: {tabela_nome}")

    # Definindo destino no LakeHouse    
    tabelas_fact_balance = (
        tabela_nome.startswith("EXP_") or
        tabela_nome.startswith("IMP_")
    )

    if tabelas_fact_balance:
        tabela_destino = re.sub(r"_\d{4}", "", tabela_nome).lower()
    else:
        tabela_destino = tabela_nome.lower()

    destino_tabela = f"{LAKEHOUSE_BASE_PATH}/{container.lower()}/{tabela_destino}"

    try:
        if container == "BALANCE":
            if nome_arquivo.endswith('.csv'):
                df_temp = spark.read.format("csv") \
                    .option("header", "true") \
                    .option("sep", ";") \
                    .option("encoding", "UTF-8") \
                    .load(path_origem)
                
                df_temp = df_temp.select(
                    [col(c).cast("string").alias(c) for c in df_temp.columns]
                )
                
                # Adiciona metadados de controle (linhagem de dados)
                df_temp = adiciona_metadados(df_temp, path_origem)

                if tabelas_fact_balance:
                    print(f"✅ Gravando FACT {tabela_nome} (append | partitionBy CO_ANO)")

                    df_temp.write \
                        .format("delta") \
                        .mode("append") \
                        .partitionBy("CO_ANO") \
                        .option("mergeSchema", "true") \
                        .save(destino_tabela)
                
                else:
                    print(f"♻️ Gravando DIM {tabela_nome} (overwrite)")

                    df_temp.write \
                        .format("delta") \
                        .mode("overwrite") \
                        .option("overwriteSchema", "true") \
                        .save(destino_tabela)
            else:
                print(f"❌ Arquivo {nome_arquivo} no {container} BALANCE não é CSV")
            
        elif container == "CNPJ":
            if nome_arquivo.endswith('.zip'):
                print(f"📦 Descompactando: {nome_arquivo}")
                arquivos_extraidos = unzip_file(path_origem, container)

                if not arquivos_extraidos:
                    print(f"⚠️ Nenhum CSV encontrado em {nome_arquivo}")
                    continue

                nome_tabela = nome_simples_tabela(nome_arquivo)

                for arquivo in arquivos_extraidos:
                    df_temp = spark.read.format("csv") \
                        .option("header", "false") \
                        .option("sep", ";") \
                        .load(path_origem)
                    
                    df_temp = renomeia_colunas(df_temp, nome_tabela)

                    df_temp = df_temp.select(
                        [col(c).cast("string").alias(c) for c in df_temp.columns]
                    )

                    # Adiciona metadados de controle (linhagem de dados)
                    df_temp = adiciona_metadados(df_temp, path_origem)

                    if nome_tabela == "empresas":
                        df_empresas.append(df_temp)
                    elif nome_tabela == "estabelecimentos":
                        df_estabelecimentos.append(df_temp)
                    elif nome_tabela == "socios":
                        df_socios.append(df_temp)
                    else:
                        df_outros.setdefault(nome_tabela, []).append(df_temp)

            else:
                print(f"❌ Arquivo {nome_arquivo} no {container} BALANCE não é ZIP")

    except Exception as e:
        print(f"❌ Erro ao processar {nome_arquivo}: {e}")

In [0]:
def union_all(dfs):
    return reduce(lambda a, b: a.unionByName(b, allowMissingColumns=True), dfs)

df_empresas_final = union_all(df_empresas) if df_empresas else None
df_estabelecimentos_final = union_all(df_estabelecimentos) if df_estabelecimentos else None
df_socios_final = union_all(df_socios) if df_socios else None

df_outros_finais = {
    nome: union_all(dfs)
    for nome, dfs in df_outros.items()
}

In [0]:
# Função simples para gravar em Delta overwrite
def write_delta_overwrite(df, destino):
    if df is None:
        return

    df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .save(destino)

base_destino = f"{LAKEHOUSE_BASE_PATH}/cnpj"

write_delta_overwrite(
    df_empresas_final,
    f"{base_destino}/empresas"
)

write_delta_overwrite(
    df_estabelecimentos_final,
    f"{base_destino}/estabelecimentos"
)

write_delta_overwrite(
    df_socios_final,
    f"{base_destino}/socios"
)

for nome_tabela, df in df_outros_finais.items():
    write_delta_overwrite(
        df,
        f"{base_destino}/{nome_tabela.lower()}"
    )